In [1]:
!pip install openai gradio fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=0699fee29d007c6e437717522fc1709ad346dc44ac5ec1bab5de44758708869d
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [2]:
import openai
import gradio as gr
import sqlite3
from fpdf import FPDF
from datetime import datetime

# Use your Groq API key here
openai.api_key = "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"
openai.api_base = "https://api.groq.com/openai/v1"


In [10]:
from openai import OpenAI
client = OpenAI(
    api_key="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",
    base_url="https://api.groq.com/openai/v1"
)

def generate_agenda(title, topics, duration):
    if not title or not topics or not duration:
        return "⚠️ Please fill in all fields correctly."

    prompt = (
        f"Create a professional meeting agenda titled '{title}'.\n"
        f"Duration: {duration} minutes.\n"
        f"Topics to cover: {topics}.\n"
        "The agenda should be clear, organized, and use bullet points."
    )

    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )
        agenda_text = response.choices[0].message.content

        # 👇 NEW: open a fresh DB connection inside the thread
        conn = sqlite3.connect("agenda.db")
        cursor = conn.cursor()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        cursor.execute(
            "INSERT INTO agendas (title, topics, agenda, timestamp) VALUES (?, ?, ?, ?)",
            (title, topics, agenda_text, timestamp)
        )
        conn.commit()
        conn.close()

        return agenda_text

    except Exception as e:
        return f"❌ Error generating agenda: {str(e)}"


In [24]:
import sqlite3

# Always do this in one cell
conn = sqlite3.connect("agenda.db")
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS agendas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        topics TEXT,
        agenda TEXT,
        timestamp TEXT
    )
""")

conn.commit()
conn.close()

print("📦 Database and 'agendas' table created.")


📦 Database and 'agendas' table created.


In [25]:
conn = sqlite3.connect("agenda.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables in database:", cursor.fetchall())
conn.close()


Tables in database: [('agendas',), ('sqlite_sequence',)]


In [26]:
def generate_agenda(title, topics, duration):
    from datetime import datetime
    import sqlite3

    if not title or not topics or not duration:
        return "⚠️ Please fill in all fields correctly."

    prompt = (
        f"Create a professional meeting agenda titled '{title}'.\n"
        f"Duration: {duration} minutes.\n"
        f"Topics to cover: {topics}.\n"
        "The agenda should be clear, organized, and use bullet points."
    )

    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )
        agenda_text = response.choices[0].message.content

        # NEW: open a local SQLite connection per request
        conn = sqlite3.connect("agenda.db")
        cursor = conn.cursor()
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        cursor.execute(
            "INSERT INTO agendas (title, topics, agenda, timestamp) VALUES (?, ?, ?, ?)",
            (title, topics, agenda_text, timestamp)
        )
        conn.commit()
        conn.close()

        return agenda_text

    except Exception as e:
        return f"❌ Error generating agenda: {str(e)}"


In [28]:

with gr.Blocks() as app:
    gr.Markdown("# 🤖 AI-Powered Meeting Agenda Generator")

    with gr.Row():
        title = gr.Textbox(label="Meeting Title", placeholder="e.g., Project Kickoff")
        topics = gr.Textbox(label="List of Topics (comma-separated)", placeholder="e.g., Introduction, Budget, Timeline")
        duration = gr.Number(label="Meeting Duration (minutes)", precision=0)

    generate_btn = gr.Button("🎯 Generate Agenda")
    agenda_output = gr.Textbox(label="Generated Agenda", lines=15)
    download_btn = gr.Button("📄 Download as PDF")
    pdf_file = gr.File(label="Download Agenda PDF", visible=False)

    # Button click event to generate agenda
    def handle_generate(title_text, topics_text, duration_value):
        result = generate_agenda(title_text, topics_text, duration_value)
        return result, gr.update(visible=False)

    # Button click event to download as PDF
    def handle_download(agenda_text, title_text):
        filename = f"{title_text.replace(' ', '_')}_agenda.pdf"
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.multi_cell(0, 10, f"Meeting Agenda - {title_text}\n\n{agenda_text}")
        pdf.output(filename)
        return gr.File.update(value=filename, visible=True)

    # Connect buttons to functions
    generate_btn.click(handle_generate, inputs=[title, topics, duration], outputs=[agenda_output, pdf_file])
    download_btn.click(handle_download, inputs=[agenda_output, title], outputs=pdf_file)

app.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1171ba9c45b963da4f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
